In [1]:
%matplotlib inline

In [2]:
import os
import sys

In [3]:
print sys.version

2.7.14 (default, Feb 15 2018, 20:22:28) 
[GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.39.2)]


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [5]:
import xgboost as xgb
import pymatgen as mg
import datetime as dt

/usr/local/lib/python2.7/site-packages/pymatgen/__init__.py:87: UserWarning: 
Pymatgen will drop Py2k support from v2019.1.1. Pls consult the documentation
at https://www.pymatgen.org for more details.
  at https://www.pymatgen.org for more details.""")


In [6]:
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [7]:
from utils import rmsle

In [8]:
from dataload import load_features

### Load data

In [9]:
DATA_DIR = './data'

In [10]:
train, test = load_features(DATA_DIR, with_ext=True, with_geo=True)

### Build train and test data sets

In [11]:
X_train = train.drop(['id', 'natoms', 'spacegroup',
                      'alpha', 'beta', 'gamma',
                      'ga', 'o_cnt', 'cellvol', 'o_fraction', 'avg_mass',
                      'bandgap', 'E0'], axis=1)
X_test = test.drop(['id', 'natoms', 'spacegroup',
                    'alpha', 'beta', 'gamma',
                    'ga', 'o_cnt', 'cellvol', 'o_fraction', 'avg_mass'], axis=1)

In [12]:
# Use log1p of energies to correct for skew
y_bg_train = train['bandgap']
y_e0_train = np.log1p(train['E0'])

In [13]:
# One-hot encode spacegroup_natoms
X_train = pd.concat([X_train.drop('spacegroup_natoms', axis=1),
                    pd.get_dummies(X_train['spacegroup_natoms'])], axis=1)
X_test = pd.concat([X_test.drop('spacegroup_natoms', axis=1),
                    pd.get_dummies(X_test['spacegroup_natoms'])], axis=1)

### Build Sklearn Model with XGBRegressor

In [23]:
param = {'learning_rate': 0.05,      # Step size shrinkage used in update (Learning rate)
         'reg_alpha': 0.01,          # L1 regularization term on weights
         'n_estimators': 700,
         'max_depth': 4,
         'subsample': 1,
         'colsample_bytree': 0.90,
         'colsample_bylevel': 0.90,
         'silent': True,
         'random_state': 42,
         'objective': 'reg:linear'}

In [24]:
# Estimator pipeline
est = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=50, random_state=42)),
    ('xgbreg', XGBRegressor(**param)),
])

In [25]:
est

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=50, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('xgbreg', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample...     reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

In [26]:
# Perform grid search for bandgap model
t_md = [3]
t_lr = [0.01, 0.02, 0.05, 0.10, 0.20]
t_ne = range(100, 700, 50)
gridsearch_bg_0 = GridSearchCV(est,
                              {'xgbreg__max_depth': t_md,
                               'xgbreg__learning_rate': t_lr,
                               'xgbreg__n_estimators': t_ne,
                              },
                              cv=5,
                              n_jobs=4,
                              scoring='neg_mean_squared_error',
                              return_train_score=False,
                              verbose=True)

In [27]:
gridsearch_bg_0.fit(X_train, y_bg_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   32.9s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  3.0min
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:  4.7min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=50, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('xgbreg', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample...     reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'xgbreg__n_estimators': [100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650], 'xgbreg__max_depth': [3], 'xgbreg__learning_rate': [0.01, 0.02, 0.05, 0.1, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring='neg_mean_squared_error', verbose=True)

In [30]:
gridsearch_bg_0.best_params_

{'xgbreg__learning_rate': 0.05,
 'xgbreg__max_depth': 3,
 'xgbreg__n_estimators': 650}

In [31]:
-gridsearch_bg_0.best_score_

0.07324532184180609

In [26]:
# Check best regularization
t_ra = [0.1, 0.05, 0.01, 0.005, 0.001]
gridsearch_bg_1 = GridSearchCV(est,
                              {'xgbreg__reg_alpha': t_ra,
                              },
                              cv=5,
                              n_jobs=4,
                              scoring='neg_mean_squared_error',
                              return_train_score=False,
                              verbose=True)

In [27]:
gridsearch_bg_1.fit(X_train, y_bg_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:   38.4s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=50, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('xgbreg', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample...     reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'xgbreg__reg_alpha': [0.1, 0.05, 0.01, 0.005, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring='neg_mean_squared_error', verbose=True)

In [28]:
gridsearch_bg_1.best_params_

{'xgbreg__reg_alpha': 0.01}

In [29]:
-gridsearch_bg_1.best_score_

0.07324532184180609

In [26]:
# Perform grid search for formation energy model
t_md = [4]
t_lr = [0.01, 0.02, 0.05, 0.10, 0.20]
t_ne = range(100, 750, 50)
gridsearch_e0_0 = GridSearchCV(est,
                              {'xgbreg__max_depth': t_md,
                               'xgbreg__learning_rate': t_lr,
                               'xgbreg__n_estimators': t_ne,
                              },
                              cv=5,
                              n_jobs=4,
                              scoring='neg_mean_squared_error',
                              return_train_score=False,
                              verbose=True)

In [27]:
gridsearch_e0_0.fit(X_train, y_e0_train)

Fitting 5 folds for each of 65 candidates, totalling 325 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   36.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  4.0min
[Parallel(n_jobs=4)]: Done 325 out of 325 | elapsed:  6.7min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=50, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('xgbreg', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample...     reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'xgbreg__n_estimators': [100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700], 'xgbreg__max_depth': [4], 'xgbreg__learning_rate': [0.01, 0.02, 0.05, 0.1, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring='neg_mean_squared_error', verbose=True)

In [28]:
gridsearch_e0_0.best_params_

{'xgbreg__learning_rate': 0.05,
 'xgbreg__max_depth': 4,
 'xgbreg__n_estimators': 700}

In [29]:
-gridsearch_e0_0.best_score_

0.0012683950744505168

In [26]:
# Check best regularization
t_ra = [0.2, 0.1, 0.05, 0.01, 0.005, 0.001]
gridsearch_e0_1 = GridSearchCV(est,
                              {'xgbreg__reg_alpha': t_ra,
                              },
                              cv=5,
                              n_jobs=4,
                              scoring='neg_mean_squared_error',
                              return_train_score=False,
                              verbose=True)

In [27]:
gridsearch_e0_1.fit(X_train, y_e0_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:  1.3min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=50, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('xgbreg', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample...     reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'xgbreg__reg_alpha': [0.2, 0.1, 0.05, 0.01, 0.005, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring='neg_mean_squared_error', verbose=True)

In [28]:
gridsearch_e0_1.best_params_

{'xgbreg__reg_alpha': 0.05}

In [29]:
-gridsearch_e0_1.best_score_

0.0012405782454539018

### Fit models using optimized hyperparameters

In [50]:
pa_bg = {'learning_rate': 0.05,      # Step size shrinkage used in update (Learning rate)
         'reg_alpha': 0.05,          # L1 regularization term on weights
         'n_estimators': 700,
         'max_depth': 3,
         'subsample': 1,
         'colsample_bytree': 0.90,
         'colsample_bylevel': 0.90,
         'silent': True,
         'random_state': 42,
         'objective': 'reg:linear'}
pa_e0 = {'learning_rate': 0.05,      # Step size shrinkage used in update (Learning rate)
         'reg_alpha': 0.1,          # L1 regularization term on weights
         'n_estimators': 800,
         'max_depth': 4,
         'subsample': 1,
         'colsample_bytree': 0.90,
         'colsample_bylevel': 0.90,
         'silent': True,
         'random_state': 42,
         'objective': 'reg:linear'}

In [51]:
# Estimator pipeline
est_bg = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=50, random_state=42)),
    ('xgbreg', XGBRegressor(**pa_bg)),
])
est_e0 = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=50, random_state=42)),
    ('xgbreg', XGBRegressor(**pa_e0)),
])

In [52]:
est_bg.fit(X_train, y_bg_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=50, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('xgbreg', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample...     reg_alpha=0.05, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

In [53]:
est_e0.fit(X_train, y_e0_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=50, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('xgbreg', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample...      reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

### Output predictions

In [54]:
predicted_bg = est_bg.predict(X_test)
predicted_e0 = np.expm1(est_e0.predict(X_test))

In [55]:
predicted = pd.DataFrame({'formation_energy_ev_natom': predicted_e0,
                          'bandgap_energy_ev': predicted_bg}) \
              .reset_index().rename(columns={'index': 'id'})
predicted['id'] += 1

In [56]:
predicted.head()

,id,bandgap_energy_ev,formation_energy_ev_natom
0,1,1.457427,0.192347
1,2,3.926876,0.081918
2,3,3.271297,0.177762
3,4,3.031129,0.038247
4,5,1.476494,0.145435


In [57]:
err_e0 = rmsle(np.expm1(est_e0.predict(X_train)), np.expm1(y_e0_train))
err_bg = rmsle(est_bg.predict(X_train), y_bg_train)

In [58]:
# Training RMSLE values
print "RMSLE BG: {}, RMSLE E0: {}, RMSLE AVG: {}".format(err_bg, err_e0,
                                                         0.5 * (err_bg + err_e0))

RMSLE BG: 0.0473847146963, RMSLE E0: 0.00884794187548, RMSLE AVG: 0.0281163282859


#### Write to file:

In [59]:
now = dt.datetime.now().strftime("%Y%m%d-%H%M%S")
predicted.to_csv(os.path.join('output', 'xgb-ext-skl-geo-pca-{}.csv'.format(now)),
                columns=['id', 'formation_energy_ev_natom', 'bandgap_energy_ev'],
                index=False)